In [3]:
import random
import os
import pandas as pd
import numpy as np
import copy
import operator

In [4]:
with open('full_dict_relations.txt', 'r') as f:
    full_dict_relations = eval(f.read())

In [43]:
# reset
list_groups = [[],[],[],[],[],[]]
last_people_base = list(full_dict_relations.keys())
last_people = list(full_dict_relations.keys())

In [6]:
# принимает номер группы, список групп. remove отвечает за удаление выбранного человека из данного списка групп. Возвращает имя этого человека
def best_person(group, last_people, remove = True):
    mid_dict = {}
    for i in last_people:
        sum_1_to_others = 0
        for j in list_groups[group - 1]:
            try:
                sum_1_to_others += full_dict_relations[i][j]
            except:
                pass
            try:
                sum_1_to_others += full_dict_relations[j][i]
            except:
                pass
        mid_dict[i] = sum_1_to_others
    sorted_dict = {k: v for k, v in sorted(mid_dict.items(), key=lambda item: item[1], reverse = True)}
    a = list(sorted_dict.keys())[0]
    if remove:
        last_people.remove(a)
    return a

In [7]:
# получает номер группы, возвращает сумму всех отношений
def sum_relation(group, list_groups):
    all_sum = 0
    for i in list_groups[group - 1]:
        for j in list_groups[group - 1]:
            try:
                all_sum += full_dict_relations[i][j]
            except:
                pass
            try:
                all_sum += full_dict_relations[j][i]
            except:
                pass
    return all_sum

In [8]:
# получает список групп, возвращает общую сумму и численность всех групп в сумме как tuple()
def sum_all_groups(list_groups):
    all_sum = 0
    population = 0
    for i in range(1, 7):
        population += len(list_groups[i - 1])
        all_sum += sum_relation(i, list_groups)
    return all_sum, population

In [9]:
# получает ключ человека в словаре отношений и номер группы.
# возвращает человека и его вклад в группу.
def contribution_one_person(person, num_group, list_groups):
    sum_contr = 0
    for i in list_groups[num_group - 1]:
        try:
            sum_contr += full_dict_relations[person][i]
        except:
            pass
        try:
            sum_contr += full_dict_relations[i][person]
        except:
            pass
    return (person, sum_contr)

In [646]:
# найти шесть лучших пятнадцаток.
# 6 групп, которые заполнены по 15 человек с максимальными суммами

In [44]:
for i in range(6):
    dict_10_groups = {}
    last_10_people = {}
    for k in last_people_base[:]:     
        list_groups[i] = []
        last_people = last_people_base[:]
        list_groups[i].append(k)
        last_people.remove(k)
        for j in range(14):  # если 14, тогда будет по 15 человек в группах.
            list_groups[i].append(best_person(i + 1, last_people, remove = True))
        a = sum_relation(i + 1, list_groups = list_groups)
        last_10_people[a] = last_people
        dict_10_groups[a] = list_groups[i]
    list_groups[i] = dict_10_groups[max(dict_10_groups)]
    last_people = last_10_people[max(dict_10_groups)]
    for people in list_groups[i]:
        last_people_base.remove(people)

In [45]:
#здесь можно проверить общую сумму, численность и оставшихся людей
sum_all_groups(list_groups = list_groups)

(2648.0, 90)

In [46]:
len(last_people)

84

In [5]:
# их заполнить с перетасовкой

In [13]:
# дать группу человека, его имя и коэф для тюна. Возвращает список рациональных переходов для этого человека.
def allowed_groups(group, name, coeff_min):
    _, current_contr = contribution_one_person(name, group, list_groups)
    minim_contrib = current_contr / coeff_min
    list_allowed_groups = []
    for i in range(6):
        _, contr = contribution_one_person(name, i + 1, list_groups)
        if contr > minim_contrib or current_contr == 0:
            list_allowed_groups.append(i + 1)
    if group not in list_allowed_groups:
        list_allowed_groups.append(group)
    return list_allowed_groups

In [14]:
# дать словарь возможных комбинаций. Возвращает количество комбинаций.
def num_comb(dictionary):
    list_comb = []
    n_comb = 1
    for key, value in dictionary.items():
        n_comb = n_comb * len(value)
    return n_comb

In [24]:
# чтобы комп не сломался на 300000 комбинациях:
# дать словарь с возможными комбинациями и список групп. Он выдаст новую комбинацию в виде нового списка групп.
def best_combination(dictionary, groups):
    n_comb = num_comb(dictionary)
    dict_comb = {}
    help_dict = {}
    new_comb = n_comb
    for key, value in dictionary.items():
        if len(value) > 1:
            new_comb = int(new_comb / len(value))
            help_dict[key] = new_comb
# очень интересная математическая штука.        
    for i in range(n_comb):
        ground_list_groups = copy.deepcopy(groups)
        for key, value in dictionary.items():
            if len(value) > 1:
                number = (i // help_dict[key]) % len(value)
                value_num = value[number]
                ground_list_groups[value_num - 1].append(key)
            else:
                ground_list_groups[value[0] - 1].append(key)
        sum_gr, _ = sum_all_groups(ground_list_groups)
        dict_comb[str(ground_list_groups)] = sum_gr
        if (i + 1) % 10000 == 0:
            print(f'{(i + 1) // 10000} iteration with {n_comb} combinations')
            dict_comb = strip_max(dict_comb)
            left_n_max(dict_comb, 1)
# если ключом оставлять сумму, тогда могут быть перезатирания, поэтому ключ - длинный список списков.
    dict_comb = strip_max(dict_comb)
    return eval(max(dict_comb, key=dict_comb.get)), n_comb

In [34]:
# удаляет все значения, где число людей в групп > 30.
def strip_max(dictionary):
    dict1 = copy.deepcopy(dictionary)
    for key, values in dictionary.items():
        for i in eval(key):
            if len(i) > 30:
                try:
                    del dict1[key]
                except:
                    pass
    return dict1

In [33]:
# оставляет в словаре n значений с наивысшими значениями
def left_n_max(d, n):
    for _ in range(len(d) - n):
        min_key = min(d.keys(), key=lambda k: d[k])
        del d[min_key]

In [36]:
# неактивен
def reorder(dict_min, list_groups, depth, coeff_min):
    for key, value in dict_min.items():
        for j in value:
            list_groups[key - 1].append(j)
    if depth > 1:
        depth = depth - 1
    else:
        coeff_min = coeff_min * 0.8
    dict_min = {}
    for group in range(6):
        dict_rel = {}
        for i in list_groups[group]:
            name, contr = contribution_one_person(i, group + 1, list_groups)
            dict_rel[name] = contr
        list_min = []
        for i in sorted(dict_rel.items(), key=lambda item: item[1])[:depth]:
            list_min.append(i[0])
        dict_min[group + 1] = list_min
        for i in list_min:
            list_groups[group].remove(i)
    dict_change_groups = {}
    for key, value in dict_min.items():
        for i in value:
            dict_change_groups[i] = allowed_groups(key, i, coeff_min)
    dict_change_groups[free] = list(range(1, 7))
    return dict_change_groups

In [ ]:
# пытался это прикрутить, чтобы группы заполнялись более равномерно, но это вызывает ошибки почему-то.
# max([len(x) for x in i]) - min([len(x) for x in i]) > 5

In [42]:
# depth - сколько мы забираем людей из каждой группы при каждой итерации
# coeff_min - насколько мы готовы уменшить вклад человека в группу, чтобы посчитать его переход в другую группу хоть частично разумным
depth = 2
coeff_min = 1
# n_group_for_free = 2 # использовал для того, чтобы свободному человеку давать не все, а три лучшие группы и первую минимальную.
# но тоже вылетали ошибки почему-то. Но это помогло бы уменьшить время перебора.

In [ ]:
# делаю излишние копии, потому что у меня уже сдвиг на связанных списках.

In [47]:
while len(last_people) > 0:
# следущий код находит по {depth} людей, которые меньше всего значат для группы. И их удаляет.
    dict_min = {}
    for group in range(6):
        dict_rel = {}
        for i in list_groups[group]:
            name, contr = contribution_one_person(i, group + 1, list_groups)
            dict_rel[name] = contr
        list_min = []
        for i in sorted(dict_rel.items(), key=lambda item: item[1])[:depth]:
            list_min.append(i[0])
        dict_min[group + 1] = list_min
        for i in list_min:
            list_groups[group].remove(i)
# для этих удалённых людей этот код создаёт словарь с возможными их переходами в другие группы.
    dict_change_groups = {}
    for key, value in dict_min.items():
        for i in value:
            dict_change_groups[i] = allowed_groups(key, i, coeff_min)
# а этот даёт новому человеку все 6 групп как возможные.
    free = last_people.pop()
    dict_change_groups[free] = list(range(1, 7))
# этот находит новую комбинацию. Далее собираются значения и выводятся на экран.
    new_list_groups, n_comb = copy.deepcopy(best_combination(dict_change_groups, copy.deepcopy(list_groups)))
    all_sum, pop = sum_all_groups(list_groups = new_list_groups)
    list_groups = copy.deepcopy(new_list_groups)
    
    for j, i in enumerate(list_groups):
        print(f"{len(i)} - {sum_relation(j, list_groups)}|", end = '')
    print(f'{all_sum} {pop} {n_comb} {all_sum / pop}')  

15 - 419.0|15 - 450.0|16 - 449.0|15 - 449.0|15 - 444.0|15 - 439.0|2650.0 91 6 29.12087912087912
15 - 419.0|15 - 450.0|17 - 449.0|15 - 453.0|15 - 444.0|15 - 439.0|2654.0 92 6 28.847826086956523
15 - 419.0|15 - 450.0|18 - 449.0|15 - 459.0|15 - 444.0|15 - 439.0|2660.0 93 6 28.602150537634408
15 - 419.0|15 - 450.0|19 - 449.0|15 - 467.0|15 - 444.0|15 - 439.0|2668.0 94 6 28.382978723404257
15 - 419.0|15 - 450.0|20 - 449.0|15 - 496.0|15 - 444.0|15 - 439.0|2697.0 95 6 28.389473684210525
15 - 419.0|15 - 450.0|21 - 449.0|15 - 518.0|15 - 444.0|15 - 439.0|2719.0 96 6 28.322916666666668
15 - 419.0|15 - 450.0|22 - 449.0|15 - 532.0|15 - 444.0|15 - 439.0|2733.0 97 6 28.175257731958762
15 - 419.0|15 - 450.0|23 - 449.0|15 - 553.0|15 - 444.0|15 - 439.0|2754.0 98 6 28.102040816326532
15 - 419.0|15 - 450.0|24 - 449.0|15 - 576.0|15 - 444.0|15 - 439.0|2777.0 99 6 28.050505050505052
15 - 419.0|15 - 450.0|25 - 449.0|15 - 596.0|15 - 444.0|15 - 439.0|2797.0 100 6 27.97
15 - 419.0|15 - 450.0|26 - 449.0|15 - 637.0

2 iteration with 20736 combinations
24 - 967.0|25 - 650.0|30 - 796.0|30 - 833.0|30 - 619.0|30 - 984.0|4849.0 169 20736 28.692307692307693
1 iteration with 20736 combinations
2 iteration with 20736 combinations
25 - 967.0|25 - 704.0|30 - 796.0|30 - 833.0|30 - 619.0|30 - 984.0|4903.0 170 20736 28.841176470588234
1 iteration with 20736 combinations
2 iteration with 20736 combinations
26 - 967.0|25 - 754.0|30 - 796.0|30 - 833.0|30 - 619.0|30 - 984.0|4953.0 171 20736 28.964912280701753
1 iteration with 20736 combinations
2 iteration with 20736 combinations
27 - 967.0|25 - 797.0|30 - 796.0|30 - 833.0|30 - 619.0|30 - 984.0|4996.0 172 20736 29.046511627906977
1 iteration with 20736 combinations
2 iteration with 20736 combinations
28 - 967.0|25 - 831.0|30 - 796.0|30 - 833.0|30 - 619.0|30 - 984.0|5030.0 173 20736 29.07514450867052
1 iteration with 20736 combinations
2 iteration with 20736 combinations
29 - 967.0|25 - 883.0|30 - 796.0|30 - 833.0|30 - 619.0|30 - 984.0|5082.0 174 20736 29.206896551

In [37]:
for i in list_groups:
    print(len(i))

30
30
29
30
30
25


In [38]:
with open('list_groups2.txt', 'w') as f:
    f.write(str(list_groups))

In [39]:
excel_groups = pd.DataFrame({'num':range(1, 31)})

In [40]:
for num, i in enumerate(list_groups):
    excel_groups[f'{num + 1} group'] = pd.Series(i)

In [41]:
excel_groups.to_excel('gen2solution.xlsx', index=None)